<a href="https://colab.research.google.com/github/so-dipe/Web-Scraping-Datasets/blob/main/UCL%20clubs%202012-2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [14]:
r = requests.get('https://www.uefa.com/uefachampionsleague/history/seasons/2022/clubs/')
soup = BeautifulSoup(r.content)

In [ ]:
clubs = soup.findAll(class_='team team-is-club')
for club in clubs:
  print(club.text.strip())

In [60]:
def get_clubs(link):
  stage_ls = []
  club_ls = []
  country_ls = []
  r = requests.get(link)
  soup = BeautifulSoup(r.content)
  fieldsets = soup.findAll('fieldset')
  
  for fieldset in fieldsets:
    legend = fieldset.find('legend').text
    teams = fieldset.findAll(class_='team team-is-club')
    for team in teams:
      club_name = team.find(attrs={'slot':'primary'}).text
      country_code = team.find(attrs={'slot':'secondary'}).text[1:-1]
      stage_ls.append(legend)
      club_ls.append(club_name)
      country_ls.append(country_code)
      
  info_dict = {'stage':stage_ls, 'club':club_ls, 'country':country_ls}
  df = pd.DataFrame(info_dict)
  return df


In [ ]:
get_clubs('https://www.uefa.com/uefachampionsleague/history/seasons/2012/clubs/')

In [91]:
def get_seasons(start, stop):
  df = pd.DataFrame()
  url = 'https://www.uefa.com/uefachampionsleague/history/seasons/' # 2012/clubs/
  start = int(start)
  stop = int(stop)
  year_range = range(start, stop + 1)
  for year in year_range:
    year_url = url + str(year) + '/clubs/'
    year_df = get_clubs(year_url)
    year_df['year'] = year
    df = pd.concat([df, year_df])
  return df

In [93]:
get_seasons(2012, 2022).to_csv('UCL clubs 2012-2022.csv', index=False)